# Todo container pode receber um IP

__Docker Object: Networking__ trata-se da capacidade de um container receber conectividade! <br>
Isto ocorre devido a uma abstração de rede feita pelos __plugins__ que a docker engine oferece, possibilitando assim modos distintos de conexões para os containers mas abstraindo todo o conceito e complexidade de rede do _host._

Sendo assim é possível para o container se comunicar com o _mundo externo,_ para tanto podemos usar os plugins default que ja vem instalados com o docker ou podemos criar os nossos próprios padrões de rede.

* __docker network ls:__ Lista os drivers networks disponiveis

## Brigde Network

É o driver de comunicação padrão. Ele cria uma " _ponte_ " entre o host e os containers, além de criar uma camada que permite ao container ter a sua própria __eth__ - interface ethernet. <br>
Aleatoriamente é criada uma subnet __privada__ que permite a cada container receber um endereço IP e se comunicar entre si e com o mundo externo.

* __Port Expose:__ Mapeamento de portas, onde possibilita que através do host possamos acessar a porta dos containers e interagir com eles. Tal mapeamento é feito através da opção __-p__ do _docker run_
    * 1 porta do host <--> 1 porta do container
    * Mapeamento Manual (-p) ou Automático (-P)
    * Exemplo: .... -p 9090:80

__OBS:__ Uma vez definida a porta usada, ela não poderá ser mais usada para nada mais!


Por que utilizar o mapeamento de portas manual ou automatico??<br>
Um bom motivo é que podemos utilizar o container no browser. Para isso utilizamos o localhost:PORTA_UTILIZADA.

Lembrando que não acessamos o __IP do container__ mas sim o __IP do host__

## Host Network

* Não há isolamento de rede entre host e container
    * Aspectos do container _continuam isolados_ como recursos, capacidade, etc..
    * Não há necessidade de mapeamento de portas pois agora o contato é direto, a mesma estrutura de rede para host e container
    
__OBS:__ Este recurso esta apenas disponível no __LINUX__ (>17.06)

* Comando: __... --network hots/--net host__

Um ponto negativo desse modo é que por usar a mesma porta do _host_ alguns containers que necessariamente utilizam a mesma porta, não irão funcionar!

## Network NONE

* Desabilita completamente a pilha de rede do container
* Isolamento total, tanto de rede como de serviço
* Frequentemente usado em casos de containers que não necessitam de comunicação externa, como por exemplo um container que faça apenas um processamento de dados
* Comando: __--network none/--net none__

## Network OVERLAY

Trabalha com ambiente Mult-Host, muito usado em _clusters_

## Network MACVLAN

É o plugin responsável por acessar as _vlans_ criadas localmente, ou seja, torna as _vlans_ acessiveis ao ambiente externo.

# Exercicios

## AÇÃO - criar e iniciar, em background, container servidor PSQL na network default padrão, mapeando a porta do host com o container na 3333, usando imagem "postgres" sugerida.

__Dicas:__ <br>

mapeamento de portas manual se dá pelo parâmetro -p do docker run. E segue a premissa porta_host:porta_container

consulte o documentação oficial da imagem no Docker Hub (IMPORTANTE). Essa imagem vem com um uso importante de variáveis de ambientes  (environmente variables) em tempo de execução fundamentais para inicialização do container - caso contrário ele não inicia corretamente. Atente-se para a variável obrigatória POSTGRES_PASSWORD. Não precisa se limitar a somente o uso da ENV obrigatória, como no caso tem a opção da ENV POSTGRES_DB.

docker run --name servidor_bridge -e POSTGRES_PASSWORD=postgres -p 3333:5432 -d postgres

## AÇÃO - criar banco de dados "teste" no container servidor PSQL usando um container psql cliente de imagem "governmentpaas/psql" sugerida.

__Dicas:__ <br>

É preciso capturar informações do IP do container servidor PSQL para estabelecer conexão entre esse dois containers, o cliente e o servidor. Lembre-se do uso do docker inspect

Aproveite o benefício de iniciar o container, com interação, removê-lo ao final do seu ciclo e executar o comando desejado em tempo de execução. Lembre-se das options --rm e -it. Para execução de comandos com parâmetros via shell você pode usar a sintaxe sh -c "COMMAND" ao final do comando docker run.

docker run --rm -it --name psql governmentpaas/psql sh -c "psql -h 172.17.0.2 -U postgres --password"

__Dentro do shell do container servidor:__ <br>
CREATE TABLE teste;

## AÇÃO - baixar e usar cliente gráfico pgAdmin no seu host, acessar servidor container PSQL e visualizar banco de dados "teste" criado, anteriormente.

Baixe o cliente pgAdmin: https://www.pgadmin.org/ no seu host e sigas orientações de uso.

* Baixado o client Postgres
* Acessa-lo através do ip local (127.0.0.1/pgadmin4) 
* Fazer login com o email e senha especificado na instalação
* criar novo server
* Em __Connections__ informar o IP local host(127.0.0.1) pois o _container servidor_ foi criado com o plugin de acesso padrão (bridge) que faz essa interface entre o container e o host
* Também informar a porta pela qual o container se comunica __3333__ que foi especificado no exercicio